In [1]:
import numpy as np
from scipy.stats import truncnorm
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.nn import MSELoss
from my_pendulum import MyPendulumEnv
from reward_ftns import pendulum_reward
from memory import TransitionMemory

In [2]:
class TransitionModel(nn.Module):
    def __init__(self, state_dim, act_dim, hidden1, hidden2):
        super(TransitionModel, self).__init__()
        self.state_dim = state_dim
        self.act_dim = act_dim
        self.fc1 = nn.Linear(state_dim + act_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, state_dim)

    def forward(self, state, act):
        x = torch.cat([state, act], dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        delta = self.fc3(x)

        next_state = state + delta  # \hat{s}_{t+1} = s_t + f(s_t, a_t ; \theta)
        
        return next_state

In [3]:
class ModelBasedAgent:
    def __init__(self,
                 state_dim,
                 act_dim,
                 ctrl_range,
                 reward_ftn,
                 hidden1=400,
                 hidden2=400,
                 lr=0.001,
                 mem_sz=20000
                 ):
        self.dimS = state_dim
        self.dimA = act_dim
        self.ctrl_range = ctrl_range
        self.model = TransitionModel(state_dim, act_dim, hidden1, hidden2)
        self.memory = TransitionMemory(state_dim, act_dim, maxlen=mem_sz)
        
        self.reward_model = reward_ftn
        
        self.optimizer = Adam(self.model.parameters(), lr=lr)
        
        self.alpha = 0.5    # smoothing parameter of cross-entropy optimization
        self.Ne = 20        # number of elite samples

    def train(self, batch_size):
        self.model.train()
        # note that training of the dynamics does not depend on any reward info
        (state_batch, act_batch, next_state_batch) = self.memory.sample_batch(batch_size)

        state_batch = torch.tensor(state_batch).float()
        act_batch = torch.tensor(act_batch).float()
        next_state_batch = torch.tensor(next_state_batch).float()

        prediction = self.model(state_batch, act_batch)


        loss_ftn = MSELoss()
        loss = loss_ftn(prediction, next_state_batch)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        loss_val = loss.detach().numpy()

        return loss_val

    def act(self, state, H, N, cross_entropy=False):
        if cross_entropy:
            return self.solve_cross_entropy_opt(state, H, N)
        else:
            return self.solve_random_shooting_opt(state, H, N)
    
    
    def solve_random_shooting_opt(self, state, H, N):
        """
        # generate K trajectories using the model of dynamics and random action sampling, and perform MPC
        # Remark! K roll-outs can be done simultaneously!

        given a state, execute an action based on random-sampling shooting method
        :param state: current state(numpy array)
        :param rew_ftn: vectorized reward function
        :param K: number of candidate action sequences to generate
        :param H: length of time horizon
        :return: action to be executed(numpy array)
        """
        assert N > 0 and H > 0

        dimA = self.dimA

        self.model.eval()

        states = np.tile(state, (N, 1)) # shape = (K, dim S)
        scores = np.zeros(N)    # array which contains cumulative rewards of roll-outs

        # generate K random action sequences of length H
        action_sequences = self.ctrl_range * (2. * np.random.rand(H, N, dimA) - 1.)
        first_actions = action_sequences[0]     # shape = (K, dim A)


        for t in range(H):
            actions = action_sequences[t]    # set of K actions, shape = (K, dim A)
            scores += self.reward_model(states, actions)

            s = torch.tensor(states).float()
            a = torch.tensor(actions).float()

            next_s = self.model(s, a)

            # torch tensor to numpy array
            # this cannot be skipped since a reward function takes numpy arrays as its inputs
            states = next_s.detach().numpy()

        best_seq = np.argmax(scores)

        return action_sequences[0, best_seq]
    
    
    
    def solve_cross_entropy_opt(self, state, H, N, rho=0.2, tol=1e-2, max_it=100):
        Ne = int(rho * N)
        dimA = self.dimA
        
        self.model.eval()

        states = np.tile(state, (N, 1)) # shape = (K, dim S)
        scores = np.zeros(N)    # array which contains cumulative rewards of roll-outs
        
        
        mu = np.zeros((H, 1, dimA))
        sigma = (.5 * self.ctrl_range) * np.ones((H, 1, dimA))
        
        assert np.max(sigma) >= tol
        best_action_so_far = None
        best_score_so_far = -np.inf
        trial = 0
        while np.max(sigma) >= tol and trial < max_it:
            action_sequences = truncnorm.rvs((-self.ctrl_range - mu) / sigma,
                                             (self.ctrl_range - mu) / sigma,
                                             loc=mu,
                                             scale=sigma,
                                             size=(H, N, dimA))
            # generate K random action sequences of length H
            for t in range(H):
                actions = action_sequences[t]    # set of K actions, shape = (K, dim A)
                scores += self.reward_model(states, actions)
                s = torch.tensor(states).float()
                a = torch.tensor(actions).float()
                next_s = self.model(s, a)
                # torch tensor to numpy array
                states = next_s.detach().numpy()
            # determine the elite samples of the group
            indices = np.argsort(scores)[-Ne:]
            idx = indices[-1]
            if scores[idx] > best_score_so_far:
                best_score_so_far = scores[idx]
                best_action_so_far = action_sequences[0, idx, :]   
            elite_samples = action_sequences[:, indices, :]
            mu += self.alpha * (np.mean(elite_samples, axis=1, keepdims=True) - mu)
            sigma += self.alpha * (np.std(elite_samples, axis=1, keepdims=True) - sigma)
            trial += 1
        if trial == max_it:
            warnings.warn('maximum iteration exceeded', RuntimeWarning)
        return best_action_so_far

In [4]:
def collect_rand_trajectories(env, transition_memory, num_trajectories):
    print('collecting random trajectories...')
    for i in range(num_trajectories):
        # collect random trajectories
        # able to be accelerated with mpi
        state = env.reset()
        for _ in range(200):
            action = env.action_space.sample()
            next_state, _, _, _ = env.step(action)
            transition_memory.append(state, action, next_state)
            state = next_state

        if i % 10 == 9:
            print('{} trajectories collected'.format(i + 1))
    print('done')
    return

In [5]:
env = MyPendulumEnv(g=10.0)
state_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]
ctrl_range = env.action_space.high[0]

In [6]:
agent = ModelBasedAgent(state_dim, act_dim, ctrl_range, pendulum_reward, hidden1=64, hidden2=64, mem_sz=30000)
collect_rand_trajectories(env, agent.memory, num_trajectories=150)

collecting random trajectories...
10 trajectories collected
20 trajectories collected
30 trajectories collected
40 trajectories collected
50 trajectories collected
60 trajectories collected
70 trajectories collected
80 trajectories collected
90 trajectories collected
100 trajectories collected
110 trajectories collected
120 trajectories collected
130 trajectories collected
140 trajectories collected
150 trajectories collected
done


In [7]:
batch_size = 256
max_iter = 50
num_ep_per_it = 5

In [8]:
# pre-train the network using randomly collected data
num_epochs = 10
epoch_size = len(agent.memory) // batch_size

for i in range(max_iter + num_epochs):
    # training loop
    # train the model
    if i == 0:
        print('pre-training...')

    if i == num_epochs:
        print('start MPC control...')

    if i < num_epochs:
        # first train the network only using randomly collected data
        for epoch in range(epoch_size):
            loss = agent.train(batch_size=batch_size)
        print('[iter {}] loss val : {:.4f}'.format(i, loss))

    else:
        scores = np.zeros(num_ep_per_it)
        
        for ep in range(num_ep_per_it):    
            state = env.reset()
            score = 0.
            for _ in range(200):
                if i % 5 == 0:
                    pass
                    # env.render()
                # environment roll-out
                # at each step, select an action using MPC (on-policy data)
                action = agent.act(state, H=60, N=400, cross_entropy=False)
                next_state, rew, _, _ = env.step(action)
                agent.memory.append(state, action, next_state)
                score += rew
                state = next_state
            scores[ep] = score
        avg = np.mean(scores)
        std = np.std(scores)
        env.close()
        print('score (over {} episodes) = {:4f}'.format(num_ep_per_it, avg), u'\u00B1', '{:4f}'.format(std))
        for _ in range(num_ep_per_it):
            loss = agent.train(batch_size=batch_size)
        print('[iter {}] loss val : {:.4f}'.format(i, loss), end='\n')
        

pre-training...
[iter 0] loss val : 0.1079
[iter 1] loss val : 0.0621
[iter 2] loss val : 0.0315
[iter 3] loss val : 0.0386
[iter 4] loss val : 0.0355
[iter 5] loss val : 0.0315
[iter 6] loss val : 0.0302
[iter 7] loss val : 0.0324
[iter 8] loss val : 0.0271
[iter 9] loss val : 0.0201
start MPC control...
score (over 5 episodes) = -326.496346 ± 183.810657
[iter 10] loss val : 0.0146
score (over 5 episodes) = -394.098863 ± 119.354604
[iter 11] loss val : 0.0249
score (over 5 episodes) = -487.992963 ± 150.517995
[iter 12] loss val : 0.0270
score (over 5 episodes) = -337.978733 ± 119.109891
[iter 13] loss val : 0.0189
score (over 5 episodes) = -638.853377 ± 269.225570
[iter 14] loss val : 0.0221
score (over 5 episodes) = -634.344600 ± 242.629409
[iter 15] loss val : 0.0198
score (over 5 episodes) = -569.558449 ± 239.194871
[iter 16] loss val : 0.0178
score (over 5 episodes) = -430.670460 ± 174.526229
[iter 17] loss val : 0.0163
score (over 5 episodes) = -608.211137 ± 180.623511
[iter 18] 